In [5]:
import pandas as pd
import numpy as np
import json

### Read politicians article data

In [7]:
politicians = pd.read_csv('page_data.csv')

### Read population data 

In [8]:
population = pd.read_csv("WPDS_2020_data.csv")